<a href="https://colab.research.google.com/github/hvhq/classificationp1/blob/master/LogisticRegression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from os import path

if not path.exists('gdrive'):
  from google.colab import drive
  drive.mount('gdrive')

Mounted at gdrive


In [2]:
#from google.colab import drive
#drive.flush_and_unmount()

In [3]:
import numpy as np
import pandas as pd
import json
import zipfile
import sklearn
import math

In [4]:
!pip install ipython-autotime
%load_ext autotime
!pip install kaggle
!mkdir -p /root/.kaggle

time: 2.43 s


In [5]:
#upload kaggle token file: kaggle.json
#files.upload()
kaggletoken={"username":"huynhvanhien","key":"82d7e5f0d5f149aedf8f040ba7c74c2a"}
with open('/root/.kaggle/kaggle.json', 'w')as kaggletokenfile:
  json.dump(kaggletoken, kaggletokenfile)
#move to /.kaggle

#!cp kaggle.json /root/.kaggle/
!chmod 600 /root/.kaggle/kaggle.json

time: 120 ms


In [6]:
!kaggle datasets download -d mohansacharya/graduate-admissions

  0% 0.00/9.64k [00:00<?, ?B/s]
100% 9.64k/9.64k [00:00<00:00, 16.5MB/s]
time: 918 ms


In [7]:
zipreference=zipfile.ZipFile('graduate-admissions.zip', 'r')
zipreference.extractall('Data')
zipreference.close()

time: 7.15 ms


In [8]:
data=pd.read_csv("Data/Admission_Predict_Ver1.1.csv")
#remove unused columns
if 'Serial No.' in data.columns:
  data=data.drop(['Serial No.'], axis=1)
#insert bias column
biasc=pd.Series([int(1) for i in range(len(data))])
#data.insert(0, "biascol", biasc, True)
data['biascol']=biasc

#filter one species
#datab=pd.DataFrame(columns=['Species','Weight','Length1','Length2', 'Length3', 'Height', 'Width'])
#for i in range(0, lendata):
#  if data.Species[i]=='Bream':
#    datab=datab.append(data.iloc[i], ignore_index=True)
#data=datab
#lendata=len(data)

#encoder string to numeric
#encoder=sklearn.preprocessing.LabelEncoder()
#encoder.fit(data.Fuel_Type)
#data.Fuel_Type=encoder.transform(data.Fuel_Type)
#encoder.fit(data.Transmission)
#data.Transmission=encoder.transform(data.Transmission)

lendata=len(data)
#change domain to [0,1]
for i in range(0, len(data.columns)):
  #print(data.iloc[:,i])
  if max(data.iloc[:,i])-min(data.iloc[:,i])!=0:
    data.iloc[:,i]=(data.iloc[:,i]-([min(data.iloc[:,i])]*lendata))/([max(data.iloc[:,i])-min(data.iloc[:,i])]*lendata)
  else:
    data.iloc[:,i]=1
print(data)

     GRE Score  TOEFL Score  ...  Chance of Admit   biascol
0         0.94     0.928571  ...          0.920635        1
1         0.68     0.535714  ...          0.666667        1
2         0.52     0.428571  ...          0.603175        1
3         0.64     0.642857  ...          0.730159        1
4         0.48     0.392857  ...          0.492063        1
..         ...          ...  ...               ...      ...
495       0.84     0.571429  ...          0.841270        1
496       0.94     0.892857  ...          0.984127        1
497       0.80     1.000000  ...          0.936508        1
498       0.44     0.392857  ...          0.619048        1
499       0.74     0.750000  ...          0.793651        1

[500 rows x 9 columns]
time: 86.7 ms


In [9]:
#split to x, y train and test
data.columns=data.columns.str.replace(' ', '_')
x=data.drop('Chance_of_Admit_', axis=1)
y=pd.DataFrame(data['Chance_of_Admit_'])
trainOTestRate=3
xtrain=x
xtest=pd.DataFrame(columns=x.columns)
ytrain=y
ytest=pd.DataFrame(columns=['Chance_of_Admit_'])
for i in range(0, len(data)):
  if i%(trainOTestRate+1)==0:
    xtest=xtest.append(x.iloc[i], ignore_index=True)
    xtrain=xtrain.drop([i])
    ytest=ytest.append(y.iloc[i], ignore_index=True)
    ytrain=ytrain.drop([i])
xtrain=xtrain.reset_index(drop=True)
ytrain=ytrain.reset_index(drop=True)

#convert y to vector
ytrain=ytrain.iloc[:,0]
ytest=ytest.iloc[:,0]
lenTrain=len(xtrain)
numberInputAttr=len(x.columns)
vector0=np.array([0]*numberInputAttr)
xtrain=np.array(xtrain)
xtest=np.array(xtest)

train
     GRE_Score  TOEFL_Score  University_Rating  ...      CGPA  Research  biascol
0         0.68     0.535714               0.75  ...  0.663462       1.0        1
1         0.52     0.428571               0.50  ...  0.384615       1.0        1
2         0.64     0.642857               0.50  ...  0.599359       1.0        1
3         0.80     0.821429               1.00  ...  0.814103       1.0        1
4         0.62     0.607143               0.50  ...  0.448718       1.0        1
..         ...          ...                ...  ...       ...       ...      ...
370       0.22     0.250000               0.50  ...  0.528846       1.0        1
371       0.84     0.571429               1.00  ...  0.711538       1.0        1
372       0.80     1.000000               1.00  ...  0.884615       1.0        1
373       0.44     0.392857               0.75  ...  0.522436       0.0        1
374       0.74     0.750000               0.75  ...  0.717949       0.0        1

[375 rows x 8 columns

In [10]:
#split train and test using ski-learn
#from sklearn.cross_validation import train_test_split
#x=data.drop('Chance_of_Admit_', axis=1)
#y=pd.DataFrame(data['Chance_of_Admit_'])
#xtrain, xtest, ytrain, ytest = train_test_split(x, y, test_size=0.3, random_state=0)

time: 1.24 ms


In [11]:
#locally weighted regression
lenTrain=len(xtrain)
alpha=0.01
numEpochs=10000

msa=0
for i in range(len(xtest)):
  theta=vector0*1
  numLoops=0
  while numLoops<numEpochs:
    error=np.dot(xtrain[numLoops%lenTrain], np.transpose(theta))-ytrain[numLoops%lenTrain]
    total_errors=np.dot(np.transpose(error), xtrain[numLoops%lenTrain])

    #local_weight=math.exp(-(xtrain[numLoops%lenTrain]-xtest)**2/2)
    xdistance=xtrain[numLoops%lenTrain]-xtest[i]
    xdistance=-xdistance * xdistance / 2
    local_weight=[math.exp(i) for i in xdistance]

    theta=theta-local_weight*(alpha*total_errors)
    numLoops+=1

  #predict:
  ypredict=np.dot(theta, xtest[i])
  #print('predict', i, ':', ypredict, 'error:',  ypredict-ytest[i])
  #print('ytest i:', ytest[i])
  msa+=abs(ypredict-ytest[i])/len(xtest)

#evaluate
print('1-msa:',1-msa)

1-msa: 0.9342532260288536
time: 32.1 s


In [45]:
#logistic regression
#batch gradient descent
#print(x)
#print(y)
theta=vector0*1

#train
alpha=0.005
numEpochs=5000
numLoops=0
while(numLoops<numEpochs):
  jtheta=np.dot(xtrain, np.transpose(theta))
  htheta=[1/(1+math.exp(-i)) for i in jtheta]
  #print(htheta)
  errors=ytrain-htheta#np.dot(xtrain, np.transpose(theta))
  sumerrors=np.dot(np.transpose(errors), xtrain)
  theta=theta+alpha*sumerrors
  numLoops+=1

#result
#print('theta: ',theta)

#test
test_jtheta=np.dot(xtest, np.transpose(theta))
test_guess=[1/(1+math.exp(-i)) for i in test_jtheta]
accuracy=sum([(test_guess[i]>=0.5)*(ytest[i]>=0.5)+(test_guess[i]<0.5)*(ytest[i]<0.5) for i in range(len(ytest))])/len(ytest)
print('accuracy:', accuracy)
mse=math.sqrt(sum([(test_guess[i]-ytest[i])**2 for i in range(len(ytest))])/len(ytest))
print("mse: ", mse)

#com_res=pd.DataFrame(test_guess, columns=['cal_result'])
#com_res.insert(1, 'check', ytest, True)
#com_res.insert(2, 'error', [1-((test_guess[i]>=0.5)*(ytest[i]>=0.5)+(test_guess[i]<0.5)*(ytest[i]<0.5)) for i in range(len(ytest))], True)


accuracy: 0.904
mse:  0.09684372475486501
time: 1.94 s


In [49]:
#try to apply locally weighted regression to logistic regression
lenTrain=len(xtrain)
alpha=0.01
numEpochs=100000
accuracy=0
msa=0
for i in range(len(xtest)):
  theta=vector0*1
  numLoops=0
  while numLoops<numEpochs:
    jtheta=np.dot(xtrain[numLoops%lenTrain], np.transpose(theta))
    htheta=1/(1+math.exp(-jtheta))
    error=ytrain[numLoops%lenTrain]-htheta
    param_errors=np.dot(np.transpose(error), xtrain[numLoops%lenTrain])

    #local_weight=math.exp(-(xtrain[numLoops%lenTrain]-xtest)**2/2)
    xdistance=xtrain[numLoops%lenTrain]-xtest[i]
    xdistance=-xdistance * xdistance / 2
    local_weight=[math.exp(i) for i in xdistance]

    theta=theta+local_weight*(alpha*param_errors)
    numLoops+=1

  #predict:
  test_jtheta=np.dot(xtest[i], np.transpose(theta))
  ypredict=1/(1+math.exp(-test_jtheta))
  #print('Predict', i, ':', ypredict, 'check:', ytest[i])
  #print('predict', i, ':', ypredict, 'error:',  ypredict-ytest[i])
  #print('ytest i:', ytest[i])
  msa+=abs(ypredict-ytest[i])/len(xtest)
  accuracy+=(ypredict>=0.5)*(ytest[i]>=0.5)+(ypredict<0.5)*(ytest[i]<0.5)
accuracy=accuracy/len(ytest)
#evaluate
print("accuracy:", accuracy)
print('1-msa:',1-msa)

accuracy: 0.888
1-msa: 0.9323723907669315
time: 55min 40s


In [47]:
#try to apply locally weighted regression to logistic regression
lenTrain=len(xtrain)
alpha=0.01
numEpochs=100000
accuracy=0
msa=0
for i in range(len(xtest)):
  theta=vector0*1
  numLoops=0
  while numLoops<numEpochs:
    jtheta=np.dot(xtrain[numLoops%lenTrain], np.transpose(theta))
    htheta=1/(1+math.exp(-jtheta))
    error=ytrain[numLoops%lenTrain]-htheta
    param_errors=np.dot(np.transpose(error), xtrain[numLoops%lenTrain])

    #local_weight=math.exp(-(xtrain[numLoops%lenTrain]-xtest)**2/2)
    xdistance=xtrain[numLoops%lenTrain]-xtest[i]
    xdistance=-xdistance * xdistance / 2
    local_weight=[math.exp(i) for i in xdistance]

    theta=theta+local_weight*(alpha*param_errors)
    numLoops+=1

  #predict:
  test_jtheta=np.dot(xtest[i], np.transpose(theta))
  ypredict=1/(1+math.exp(-test_jtheta))
  #print('Predict', i, ':', ypredict, 'check:', ytest[i])
  #print('predict', i, ':', ypredict, 'error:',  ypredict-ytest[i])
  #print('ytest i:', ytest[i])
  msa+=abs(ypredict-ytest[i])/len(xtest)
  accuracy+=(ypredict>=0.5)*(ytest[i]>=0.5)+(ypredict<0.5)*(ytest[i]<0.5)
accuracy=accuracy/len(ytest)
#evaluate
print("accuracy:", accuracy)
print('1-msa:',1-msa)

accuracy: 0.904
1-msa: 0.9326085778283871
time: 5min 33s


In [ ]:
#newton's method for gradient descent
